In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from warnings import filterwarnings
%matplotlib notebook

In [2]:
filterwarnings(action="ignore")

In [3]:
r2 = lambda x: round(x,2)
r3 = lambda x: round(x,3)

# Get the data

In [4]:
chd_df = pd.read_csv("framingham.csv")

### Inspect Dataframe

In [5]:
chd_df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


### Understand Features/Variables

#### Columns/Variables Explained:
* **sex**: male or female (male=1)
* **age**: age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)
* **education**: 1 = Some High School; 2 = High School or GED; 3 = Some College or Vocational School; 4 = college (Categorical - Nominal)
* **currentSmoker**: whether or not the patient is a current smoker (current smoker=1)
* **cigsPerDay**: the number of cigarettes that the person smoked on average in one day.
* **BPMeds**: whether or not the patient was on blood pressure medication (on BP medication = 1)
* **prevalentStroke**: whether or not the patient had previously had a stroke (previous stroke = 1)
* **prevalentHyp**: whether or not the patient was hypertensive (hypertensive = 1)
* **diabetes**: whether or not the patient had diabetes (diabetes = 1)
* **totChol**: total cholesterol level.
* **sysBP**: systolic blood pressure.
* **diaBP**: diastolic blood pressure.
* **BMI**: Body Mass Index.
* **heartRate**: heart rate.
* **glucose**: glucose level
* **TenYearCHD**: 10 year risk of coronary heart disease CHD (yes=1)

### Inspect datatypes and null values

In [6]:
chd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   int64  
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 530.1 KB


### Check how many null values are in each row

In [7]:
chd_df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

### Percent of null values

In [8]:
r2((chd_df.isnull().sum() / chd_df.count()) * 100)

male                0.00
age                 0.00
education           2.54
currentSmoker       0.00
cigsPerDay          0.69
BPMeds              1.27
prevalentStroke     0.00
prevalentHyp        0.00
diabetes            0.00
totChol             1.19
sysBP               0.00
diaBP               0.00
BMI                 0.45
heartRate           0.02
glucose            10.07
TenYearCHD          0.00
dtype: float64

### Drop null rows

In [9]:
# drops rows that have a null value
chd_df = chd_df.dropna(how="any").reset_index(drop=True)

In [10]:
# percent of null values should be 0.0 for all rows since null values were dropped
(chd_df.isnull().sum() / chd_df.count()) * 100

male               0.0
age                0.0
education          0.0
currentSmoker      0.0
cigsPerDay         0.0
BPMeds             0.0
prevalentStroke    0.0
prevalentHyp       0.0
diabetes           0.0
totChol            0.0
sysBP              0.0
diaBP              0.0
BMI                0.0
heartRate          0.0
glucose            0.0
TenYearCHD         0.0
dtype: float64

### Percent of observations with TenYearCHD

In [11]:
pctTYchd = (chd_df.TenYearCHD.sum() / chd_df.shape[0]) * 100

In [12]:
print("Percentage of observations with TenYearCHD risk: {:.2f}%".format(pctTYchd))

Percentage of observations with TenYearCHD risk: 15.23%


Since only 15.23% of our observations have TenYearCHD risk there maybe some misclassification on positive values.

### Encode the categorical - nominal variable education

In [13]:
chd_df = pd.get_dummies(chd_df,columns=["education"],drop_first=True)

### Place independent variable at the end of the matrix

In [14]:
tyd = chd_df.pop("TenYearCHD")

In [15]:
chd_df.insert(loc=len(chd_df.columns),column="TenYearCHD",value = tyd)

### Separate independent and dependent variables 

In [16]:
X = chd_df.drop("TenYearCHD",axis=1) # Dataframe of features
y = chd_df.iloc[:,-1]# independent variable vector

### Test for statistical Significance
 - alpha = 0.05
 - pvalue < alpha

In [17]:
logit = sm.Logit

In [18]:
results = logit(y,X).fit()

Optimization terminated successfully.
         Current function value: 0.396146
         Iterations 6


In [19]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3658
Model:                          Logit   Df Residuals:                     3641
Method:                           MLE   Df Model:                           16
Date:                Thu, 02 Jun 2022   Pseudo R-squ.:                 0.07144
Time:                        14:24:42   Log-Likelihood:                -1449.1
converged:                       True   LL-Null:                       -1560.6
Covariance Type:            nonrobust   LLR p-value:                 1.723e-38
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
male                0.3688      0.105      3.501      0.000       0.162       0.575
age                 0.0266      0.006      4.509      0.000       0.015       0.038
currentSmoker      -0.2095      0.153     -1.371      0.170      -0.509       0.090
cigsPerDay          0.0209      0.006      3.378      0.001       0.009       0.033
BPMeds              0.3716      0.231      1.611      0.107      -0.081       0.824
prevalentStroke     0.7298      0.489      1.493      0.135      -0.228       1.688
prevalentHyp        0.9253      0.124      7.444      0.000       0.682       1.169
diabetes            0.7197      0.298      2.413      0.016       0.135       1.304
totChol            -0.0010      0.001     -0.931      0.352      -0.003       0.001
sysBP               0.0130      0.004      3.420      0.001       0.006       0.020
diaBP              -0.0265      0.006     -4.324      0.000      -0.038      -0.014
BMI                -0.0503      0.012     -4.081      0.000      -0.074      -0.026
heartRate          -0.0221      0.004     -5.548      0.000      -0.030      -0.014
glucose             0.0021      0.002      1.013      0.311      -0.002       0.006
education_2.0      -0.4614      0.118     -3.897      0.000      -0.693      -0.229
education_3.0      -0.4296      0.145     -2.959      0.003      -0.714      -0.145
education_4.0      -0.3396      0.159     -2.130      0.033      -0.652      -0.027
===================================================================================
"""

### Backwards Elimination

   - Iterate through features and remove insignificant features and retest the model

In [20]:
class BackwardElimination:
    
    """
    Backwards Elimination:
        
        Gradually eliminates insignificant variables.
        
        Parameters
        ----------     
        X: A pandas.DataFrame of independent variables.
        
        y: The dependent variable.
        
        sl: alpha or significance level. Default is 0.05.
    """
    
    def __init__(self,X,y,sl=0.05):
        self.X = X
        self.y = y
        self.sl = sl
        self.sigCols = ''
        
    def backward_elimination(self):
        
        colList = self.X.columns
    
        while len(colList) > 0:
            model = sm.Logit(self.y,self.X[colList])
            result = model.fit()
            largest_pvalue = r3(result.pvalues).nlargest(1)
            if largest_pvalue[0] < self.sl:
                self.sigCols = colList
                return result
                break
            else:
                colList = colList.drop(largest_pvalue.index)

In [21]:
be = BackwardElimination(X,y)

In [22]:
resultsBE = be.backward_elimination()

Optimization terminated successfully.
         Current function value: 0.396146
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.396265
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.396396
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.396656
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.396949
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.397367
         Iterations 6


In [23]:
resultsBE.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3658
Model:                          Logit   Df Residuals:                     3646
Method:                           MLE   Df Model:                           11
Date:                Thu, 02 Jun 2022   Pseudo R-squ.:                 0.06858
Time:                        14:24:42   Log-Likelihood:                -1453.6
converged:                       True   LL-Null:                       -1560.6
Covariance Type:            nonrobust   LLR p-value:                 8.956e-40
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
male              0.3713      0.104      3.558      0.000       0.167       0.576
age               0.0257      0.006      4.577      0.000       0.015       0.037
cigsPerDay        0.0141      0.004      3.400      0.001       0.006       0.022
prevalentHyp      0.9675      0.121      8.021      0.000       0.731       1.204
diabetes          0.9240      0.230      4.009      0.000       0.472       1.376
sysBP             0.0137      0.004      3.632      0.000       0.006       0.021
diaBP            -0.0273      0.006     -4.482      0.000      -0.039      -0.015
BMI              -0.0496      0.012     -4.093      0.000      -0.073      -0.026
heartRate        -0.0234      0.004     -6.096      0.000      -0.031      -0.016
education_2.0    -0.4699      0.117     -4.001      0.000      -0.700      -0.240
education_3.0    -0.4312      0.144     -2.994      0.003      -0.713      -0.149
education_4.0    -0.3577      0.159     -2.255      0.024      -0.669      -0.047
=================================================================================
"""

In [24]:
be.sigCols # Statistically significant features

Index(['male', 'age', 'cigsPerDay', 'prevalentHyp', 'diabetes', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'education_2.0', 'education_3.0',
       'education_4.0'],
      dtype='object')

# Logistic Regression

   - Using statistically significant features

In [25]:
X = chd_df[be.sigCols].values
y = chd_df.iloc[:,-1].values

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=3)

In [27]:
lrm = LogisticRegression()

In [28]:
lrm.fit(X_train,y_train)

LogisticRegression()

In [29]:
yPred = lrm.predict(X_test)

### Confusion Matrix

 - a table that is used to define the performance of a classification algorithm

In [30]:
cm = confusion_matrix(y_test,yPred)

In [31]:
print("Confusion Matrix:")

cm

Confusion Matrix:


array([[773,  15],
       [115,  12]])

In [32]:
tn,fp,fn,tp = cm.ravel()

In [33]:
singleModelAcc = accuracy_score(y_test,yPred) * 100

In [34]:
print("Model Accuracy: {:.2f}%".format(singleModelAcc))

Model Accuracy: 85.79%


In [35]:
r2((tp+tn) / cm.sum() * 100) # Calculating model accuracy manually

85.79

# Type I & Type II Error

- Type I Error
    - Mistaken rejection of an actually true null hypothesis (False Positive)
    
- Type II Error
    - the failure to reject a null hypothesis that is actually false (False Negative)
    
    *Since we are dealing with Predicting heart disease, Higher value of false negatives is dangerous. 
    False Negative = There is Heart Disease, but we predict it wrongly as No Heart Disease. *

In [36]:
tn,fp,fn,tp =  cm.ravel()

In [37]:
fp # Type I error

15

In [38]:
fn # type two error

115

### Model Sensitivity

- Sensitivity (true positive rate) refers to the probability of a positive test, conditioned on truly being positive.

In [39]:
sense = tp / (tp+fn) * 100

print("Sensitivity {:.2f}%".format(sense))

Sensitivity 9.45%


### Model Specificity
- Specificity (true negative rate) refers to the probability of a negative test, conditioned on truly being negative.

In [40]:
spec = tn / (tn + fp) * 100

print("Specificity {:.2f}%".format(spec))

Specificity 98.10%


In [41]:
singleModelAcc = accuracy_score(y_test,yPred) * 100

# Cross Validation
- a resampling method that uses different portions of the data to test and train a model on different iterations

In [42]:
lrm = LogisticRegression(C=10) # parameter tuning

In [43]:
cross_val_score(lrm,X,y,cv=10)

array([0.8579235 , 0.84972678, 0.83879781, 0.84699454, 0.84153005,
       0.84972678, 0.84972678, 0.84153005, 0.85205479, 0.84657534])

In [44]:
tenFoldCVscore = cross_val_score(lrm,X,y,cv=10).mean() * 100

In [45]:
print("10 Fold Cross Validation: {:.2f}%".format(tenFoldCVscore))

10 Fold Cross Validation: 84.75%


# Logistic Regression Scaling Features

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=3)

In [47]:
x_sc = StandardScaler()

In [48]:
X_train = x_sc.fit_transform(X_train)
X_test = x_sc.fit_transform(X_test)

In [49]:
lrm.fit(X_train,y_train)

LogisticRegression(C=10)

In [50]:
yPred = lrm.predict(X_test)

In [51]:
cm2 = confusion_matrix(y_test,yPred)

In [52]:
tn, fp, fn, tp = cm.ravel()

In [53]:
sense2 = tp / (tp+fn) * 100

In [54]:
print("Sensitivity {:.2f}%".format(sense2))

Sensitivity 9.45%


In [55]:
spec2 = tn/(tn+fp) * 100

In [56]:
print("Specificity {:.2f}%".format(spec2))

Specificity 98.10%


In [57]:
maSF = accuracy_score(y_test,yPred) * 100

print("Model Accuracy: {:.2f}%".format(maSF))

Model Accuracy: 86.61%


# Cross Validation With Scaled Features

In [58]:
lrm = LogisticRegression()

In [59]:
X2 = x_sc.fit_transform(X)

In [60]:
print("Cross Val Scaled Features: {}".format(r2(cross_val_score(lrm,X2,y).mean() * 100)))

Cross Val Scaled Features: 85.05


# Resampling to Reduce Type II Error

     - Due to the low number of observations that have 10 year CHD 15% 

In [61]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=3)

In [62]:
smo = SMOTE()

In [63]:
X_train, y_train = smo.fit_resample(X_train, y_train)

In [64]:
lrm.fit(X_train, y_train)

LogisticRegression()

In [65]:
yPred = lrm.predict(X_test)

In [66]:
cmRE = confusion_matrix(y_test,yPred)

In [67]:
print("Confusion Matrix:")
cmRE

Confusion Matrix:


array([[531, 257],
       [ 52,  75]])

In [68]:
tn,fp,fn,tp = cmRE.ravel()

In [69]:
sense = tp / (tp+fn) * 100

print("Sensitivity {:.2f}%".format(sense))

Sensitivity 59.06%


In [70]:
spec = tn / (tn + fp) * 100

print("Specificity {:.2f}%".format(spec))

Specificity 67.39%


In [71]:
rsModelAcc = accuracy_score(y_test,yPred) * 100

In [72]:
print("Logistic Model Accuracy: {:.2f}%".format(rsModelAcc))

Logistic Model Accuracy: 66.23%
